In [1]:
%matplotlib inline
    
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split

from sklearn import model_selection
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import accuracy_score 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
# final df for model building
df1 = pd.read_csv("ML_model_wk40_to_20.csv")
df1.dropna(how='any', inplace=True)
del df1['ILI_weeks']
del df1['Unnamed: 0']
#del df1['week']
df1.head(2)

,Year,week,a_influenza,acute_bronchitis,body_temperature,braun_thermoscan,break_a_fever,bronchitis,chest_cold,cold_and_flu,...,walking_pneumonia,what_to_do_if_you_have_the_flu,Flu_Visit_Count,ILI_Visit_Count,Unspecified,CDC_Unweighted_ILI,ILI_lagwk1,ILI_lagwk2,ILI_lagwk3,ILI_lagwk4
0,2009,40,44,33,81,69,22,40,35,36,...,47,56,0.01338,0.01763,0.03074,5.66087,6.81522,7.61889,7.38836,6.33927
1,2009,41,51,51,77,46,24,43,35,43,...,53,58,0.01620,0.02103,0.03554,6.81522,7.61889,7.38836,6.33927,4.94349


In [4]:
#  combining year and week column 
def combine_year_week(row):
    return int(row["Year"]) * 100 + int(row["week"])

In [5]:
df1["YearWeek"] = df1.apply(combine_year_week, axis=1)

In [6]:
# reset index to YearWeek
df1.index = df1["YearWeek"]

In [7]:
model_df = df1.drop(["Year", "week", "YearWeek"], axis=1)
model_df.head(2)

,a_influenza,acute_bronchitis,body_temperature,braun_thermoscan,break_a_fever,bronchitis,chest_cold,cold_and_flu,cold_or_flu,cold_versus_flu,...,walking_pneumonia,what_to_do_if_you_have_the_flu,Flu_Visit_Count,ILI_Visit_Count,Unspecified,CDC_Unweighted_ILI,ILI_lagwk1,ILI_lagwk2,ILI_lagwk3,ILI_lagwk4
YearWeek,,,,,,,,,,,,,,,,,,,,,
200940,44,33,81,69,22,40,35,36,37,30,...,47,56,0.01338,0.01763,0.03074,5.66087,6.81522,7.61889,7.38836,6.33927
200941,51,51,77,46,24,43,35,43,49,41,...,53,58,0.01620,0.02103,0.03554,6.81522,7.61889,7.38836,6.33927,4.94349


# Independent variables
        # 3 independent variables from athena EHR 
            # [(flu visit counts)/ (total patient visit counts) 
            # (ILI visit counts)/ (total patient visit counts)
            # (unspecified viral or ILI visit counts)/ (total patient visit counts)]
        # CDC historical CDC_Unweighted_ILI values: collected from 2009 to 2016 (week 40 to 20)
        # 129 google search terms related to flu
        
      ======> 3 + 1 + 129
      
# Dependent variables 
        # 4 ILI weeks offset by 1 week

# split df to train dataset, dataset before year2015 and week 40 used at training dataset

In [8]:
train = model_df[model_df.index < 201540]
# train.head()


# defining  targets/labels to  y axis
                y_train = ILI lag week 1

                yy_train = ILI lag week 2

                yyy_train = ILI lag week 3

                yyyy_train = ILI lag week 4

In [9]:
y_train = train["ILI_lagwk1"]
# y_train

In [10]:
yy_train = train["ILI_lagwk2"]

In [11]:
yyy_train = train["ILI_lagwk3"]

In [12]:
yyyy_train =train["ILI_lagwk4"]

In [13]:
X_train = train.drop(["ILI_lagwk1", "ILI_lagwk2", "ILI_lagwk3", "ILI_lagwk4"], axis=1)
# X_train

# split df to train dataset,dataset after year2015 and week 40 used at training dataset

In [14]:
test = model_df[model_df.index >= 201540]
# test.head()

In [15]:
y_test = test["ILI_lagwk1"]

In [16]:
yy_test = test["ILI_lagwk2"]

In [17]:
yyy_test = test["ILI_lagwk3"]

In [18]:
yyyy_test = test["ILI_lagwk4"]

In [19]:
X_test = test.drop(["ILI_lagwk1", "ILI_lagwk2", "ILI_lagwk3", "ILI_lagwk4"], axis=1)

# Reference https://shankarmsy.github.io/stories/gbrt-sklearn.html#

##https://www.youtube.com/watch?v=IXZKgIsZRm0

# building a model for y_test = test["ILI_lagwk1"]

In [174]:
knn1 = KNeighborsRegressor(n_neighbors = 50, leaf_size = 60, algorithm = 'auto')
# )
#, algorithm='auto', )  
# algorithm =  auto, brute, kd_tree, ball_tree, 

In [175]:
knn1.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=60, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=50, p=2,
          weights='uniform')

In [176]:
y_pred = knn1.predict(X_test)

In [177]:
r2_score(y_test, y_pred)

0.6886005429839699

In [178]:
mean_absolute_error(y_test, y_pred)

0.2571126484848484

In [179]:
print("R-squared for ILI_lagwk1 Train: %.2f" %knn1.score(X_train, y_train)) 
print("R-squared for ILI_lagwk1 Test : %.2f" %knn1.score(X_test, y_test)) 

R-squared for ILI_lagwk1 Train: 0.63
R-squared for ILI_lagwk1 Test : 0.69


In [78]:
# predict the values of y
predictions1 = knn1.predict(X_test) 
y_test_unraveled = y_test.values.ravel()
y_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [79]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk1": predictions1, "Actual": y_test_unraveled}).reset_index(drop=True)
pred_df.index = y_test.index
pred_df.head()

,Prediction_ILI_lagwk1,Actual
YearWeek,,
201540,1.345412,1.33029
201541,1.465838,1.41368
201542,1.776847,1.36882
201543,1.709732,1.48309
201544,1.709732,1.54494


# # Create the GridSearchCV model

In [80]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {'learning_rate':[ 0.02, 0.01, 0.1, 0.05], 
#             'max_depth':[1, 3, 5 ,7], 
#             'min_samples_leaf':[3, 5, 7 ,9], 
# #             'max_features':[0.1,0.3,1.0],
#             'n_estimators': [300, 500, 1000, 2000]
#              } 

# est = GradientBoostingRegressor()
# gs_cv = GridSearchCV(est, param_grid).fit(X_train, y_train)

# # best hyperparameter setting

# gs_cv.best_est 

# selecting important features  ["ILI_lagwk1"]

In [81]:
# # GBRF predictors variable importance
# # store most important variables under importances
# importances = gbrt.feature_importances_

In [82]:
# gbrt.feature_importances_.sum()

In [83]:
# # store most important variables/features under importances
# importances = gbrt.feature_importances_
# # importances

In [84]:
# sorted_features = sorted(zip(importances, X_train.columns), reverse=True)
# # sorted_features

In [85]:
# # sort important varibles and sotre them under indices
# important_columns = []
# for fimportance, name in sorted(zip(importances, X_train.columns), reverse=True):
#     if fimportance > 0.00009:
#         important_columns.append(name)

In [86]:
# important_feature_columns = important_columns
# # important_feature_columns

In [87]:
# important_features_df = model_df[important_feature_columns]
# important_features_df.head()

# defining important feature X train and X test dataset

In [88]:
# X_trainim = important_features_df[important_features_df.index < 201540]
# # trainim
# # .head(2)

In [89]:
# X_testim = important_features_df[important_features_df.index >= 201540]
# # testim
# # .head()

In [90]:
# gbrtim_ili_lag1 = GradientBoostingRegressor(n_estimators = 500, max_depth = 5) # number of sequential trees to be modeled

In [91]:
# gbrtim_ili_lag1.fit(X_trainim, y_train) 

In [92]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [93]:
# r2_score(y_test, y_pred)

In [94]:
# mean_absolute_error(y_test, y_pred)

In [95]:
# # GBR with important features for ili_lag1 week 1
# print("R-squared for Train gbrtim_ili_lag1: %.2f" %gbrtim_ili_lag1.score(X_trainim, y_train)) 
# print("R-squared for Test gbrtim_ili_lag1: %.2f" %gbrtim_ili_lag1.score(X_testim, y_test)) 

In [96]:
# # predict the values of y with important feature extraction for ili week 1
# predictions = gbrtim_ili_lag1.predict(X_testim)
# y_test_unraveled = y_test.values.ravel()
# y_test.index

In [97]:
# Make predictions using the X_test and y_test data
# Print at least 10 predictions vs their actual labels
# predictions = gbrtim_ili_lag1.predict(X_testim)
# print(f"First 10 Predictions: {predictions[:10]}")
# print(f"First 10 Actual labels: {y_test_unraveled[:10]}")

In [98]:
# # Print predictions vs their actual labels
# pred_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test_unraveled}).reset_index(drop=True)
# pred_df.index = y_test.index
# pred_df.head()

# building a model for y_test = test["ILI_lagwk2"]

In [99]:
knn2 = KNeighborsRegressor(n_neighbors = 6)#, leaf_size = 20) 

In [100]:
knn2.fit(X_train, yy_train) 

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=6, p=2,
          weights='uniform')

In [101]:
yy_pred = knn2.predict(X_test)

In [102]:
r2_score(yy_test, yy_pred)

0.2465196154638276

In [103]:
mean_absolute_error(yy_test, yy_pred)

0.38958949494949496

In [104]:
print("R-squared for ILI_lagwk2 Train: %.2f" %knn2.score(X_train, yy_train)) 
print("R-squared for ILI_lagwk2 Test: %.2f" %knn2.score(X_test, yy_test)) 

R-squared for ILI_lagwk2 Train: 0.80
R-squared for ILI_lagwk2 Test: 0.25


In [106]:
predictions2 = knn2.predict(X_test) 

In [107]:
# predict the values of y
yy_test_unraveled = yy_test.values.ravel()
yy_test.index

Int64Index([201540, 201541, 201542, 201543, 201544, 201545, 201546, 201547,
            201548, 201549, 201550, 201551, 201552, 201601, 201602, 201603,
            201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611,
            201612, 201613, 201614, 201615, 201616, 201617, 201618, 201619,
            201620],
           dtype='int64', name='YearWeek')

In [108]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk2": predictions2, "Actual": yy_test_unraveled}).reset_index(drop=True)
pred_df.index = yy_test.index
pred_df.head()

,Prediction_ILI_lagwk2,Actual
YearWeek,,
201540,1.411195,1.41368
201541,1.537820,1.36882
201542,2.055755,1.48309
201543,1.889823,1.54494
201544,1.889823,1.62532


In [109]:
# # Create the GridSearchCV model

# from sklearn.model_selection import GridSearchCV
# param_grid = {'learning_rate':[ 0.02, 0.01, 0.1, 0.05], 
#             'max_depth':[1, 3, 5 ,7], 
#             'min_samples_leaf':[3, 5, 7 ,9], 
# #             'max_features':[0.1,0.3,1.0],
#             'n_estimators': [300, 500, 1000, 2000]
#              } 

# est = GradientBoostingRegressor()
# gs_cv = GridSearchCV(est, param_grid).fit(X_train, y_train)

# # best hyperparameter setting

# gs_cv.best_est 

# selecting important features  ["ILI_lagwk2"]

# defining important feature X train and X test dataset

# building a model for y_test = test["ILI_lagwk3"]

In [110]:
knn3 = KNeighborsRegressor(n_neighbors = 6)#, leaf_size = 20)  

In [111]:
knn3.fit(X_train, yyy_train) 

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=6, p=2,
          weights='uniform')

In [112]:
yyy_pred = knn3.predict(X_test)

In [113]:
r2_score(yyy_test, yyy_pred)

0.04261510412855207

In [114]:
mean_absolute_error(yyy_test, yyy_pred)

0.46994661616161626

In [115]:
print("R-squared for Train: %.2f" %knn3.score(X_train, yyy_train)) 
print("R-squared for Test: %.2f" %knn3.score(X_test, yyy_test)) 

R-squared for Train: 0.74
R-squared for Test: 0.04


In [69]:
predictions3 = knn3.predict(X_test) 

In [ ]:
# predict the values of y
predictions = knn3.predict(X_test)
yyy_test_unraveled = yyy_test.values.ravel()
yyy_test.index

In [ ]:
pred_df = pd.DataFrame({"Prediction_ILI_lagwk3": predictions3, "Actual": yyy_test_unraveled}).reset_index(drop=True)
pred_df.index = yyy_test.index
pred_df.head()

# building a model for y_test = test["ILI_lagwk4"]

In [ ]:
knn4 = KNeighborsRegressor(n_neighbors = 6)#, leaf_size = 20)

In [ ]:
knn4.fit(X_train, yyyy_train) 

In [ ]:
yyyy_pred = knn4.predict(X_test)

In [ ]:
r2_score(yyyy_test, yyyy_pred)

In [ ]:
mean_absolute_error(yyyy_test, yyyy_pred)

In [ ]:
print("R-squared for Train: %.2f" %knn4.score(X_train, yyyy_train)) 
print("R-squared for Test: %.2f" %knn4.score(X_test, yyyy_test)) 

In [ ]:
# predict the values of y
predictions4 = knn4.predict(X_test)
yyyy_test_unraveled = yyyy_test.values.ravel()
yyyy_test.index

In [ ]:
# store prediction and actual to df
pred_df = pd.DataFrame({"Prediction_ILI_lagwk1": predictions4, "Actual": yyyy_test_unraveled}).reset_index(drop=True)
pred_df.index = yyyy_test.index
pred_df.head()